## Notebook inspired by the Quantifying Uncertainty in Marathon Finish Time Prediction work here: 
https://www.researchgate.net/publication/385855387_Quantifying_Uncertainty_in_Marathon_Finish_Time_Predictions

In [1]:
from src.pyrox.core import PyroxClient
from src.pyrox.reporting import ReportingClient

In [2]:
client = PyroxClient()
report_client = ReportingClient()

In [3]:
s8 = client.get_season(season=8)

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Dict, Iterable, List, Optional, Sequence, Tuple

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# segment order, after each segment train a model to predict remaining time
SEGMENT_ORDER: List[str] = [
    "run1_time",
    "skiErg_time",
    "run2_time",
    "sledPush_time",
    "run3_time",
    "sledPull_time",
    "run4_time",
    "burpeeBroadJump_time",
    "run5_time",
    "rowErg_time",
    "run6_time",
    "farmersCarry_time",
    "run7_time",
    "sandbagLunges_time",
    "run8_time",
    "wallBalls_time",
]

# Categorical context
CAT_COLS = ["division", "gender", "age_group", "event_id"]

# Total time (target is always based on total_time)
TOTAL_COL = "total_time"

ROXZONE_COL = "roxzone_time"


# -----------------------------
# 1) Helper: parse times safely
# -----------------------------
def _to_seconds(s: pd.Series) -> pd.Series:
    """
    Convert time columns to seconds.

    - If already numeric, keep as float.
    - If strings like "01:04:35" or "64:35", pandas.to_timedelta handles both.
    """
    if pd.api.types.is_numeric_dtype(s):
        return s.astype(float)
    td = pd.to_timedelta(s, errors="coerce")
    return td.dt.total_seconds()


def _z_for_central_interval(central: float) -> float:
    """
    Convert a central interval (e.g. 0.80) into a z-score under a Normal assumption.

    central=0.80 => want 10th..90th percentiles.
    z = norm.ppf(0.5 + central/2)

    We avoid importing scipy for a tiny utility and use common defaults.
    If you want exact z, you can use scipy.stats.norm.ppf. :contentReference[oaicite:3]{index=3}
    """
    if abs(central - 0.80) < 1e-9:
        return 1.2815515655446004  # ~ norm.ppf(0.9) :contentReference[oaicite:4]{index=4}
    if abs(central - 0.90) < 1e-9:
        return 1.6448536269514722
    if abs(central - 0.95) < 1e-9:
        return 1.959963984540054
    # Fallback (reasonable default)
    return 1.2815515655446004



@dataclass(frozen=True)
class CheckpointModel:
    """
    A single checkpoint model:

    Given splits up to checkpoint t, predict remaining time r_t.

    We keep:
    - preprocessor: standardize numeric + one-hot categoricals
    - model: BayesianRidge (returns predictive std)
    - segments_used: the prefix of SEGMENT_ORDER used at this checkpoint
    """
    preprocessor: ColumnTransformer
    model: BayesianRidge
    segments_used: Tuple[str, ...]
    use_roxzone: bool

    def predict_finish_seconds(
        self,
        row: pd.Series,
        central_interval: float = 0.80,
    ) -> Tuple[float, float, float]:
        """
        Predict finish time with a simple Normal-ish interval:

        finish_mean = c_t + r_mean
        finish_low/high = finish_mean +/- z * r_std

        BayesianRidge gives r_mean and r_std via predict(return_std=True). :contentReference[oaicite:5]{index=5}
        """
        row = row.copy()


        c_t = float(row[list(self.segments_used)].sum())
        row["c_t"] = c_t

        row["n_segments"] = len(self.segments_used)

        num_cols = list(self.segments_used) + ["c_t", "n_segments"]
        if self.use_roxzone and ROXZONE_COL in row.index:
            num_cols.append(ROXZONE_COL)

        X_raw = pd.DataFrame([row[num_cols + CAT_COLS]])

        X = self.preprocessor.transform(X_raw)
        r_mean, r_std = self.model.predict(X, return_std=True)  

        r_mean = float(r_mean[0])
        r_std = float(r_std[0])

        finish_mean = c_t + r_mean

        z = _z_for_central_interval(central_interval)
        finish_low = finish_mean - z * r_std
        finish_high = finish_mean + z * r_std

        return finish_mean, finish_low, finish_high


# -----------------------------
# 3) The main utility class
# -----------------------------
class HyroxBayesianLivePredictor:
    """
    Trains Bayesian checkpoint models for HYROX.

    How it works (simple):
    - For each checkpoint t = 1..16
      - segments_used = SEGMENT_ORDER[:t]
      - label r_t = total_time - sum(segments_used)
      - train BayesianRidge to predict r_t from:
        * the observed segments so far
        * simple aggregates like c_t
        * context (division/gender/age_group/event_id)

    Then at runtime:
    - You call predict_live(row, completed_segments)
    - It chooses the right checkpoint model and outputs (mean, low, high).
    """

    def __init__(
        self,
        *,
        central_interval: float = 0.80,
        use_roxzone: bool = True,
        test_size: float = 0.2,
        random_state: int = 7,
        min_rows_per_checkpoint: int = 500,
    ) -> None:
        self.central_interval = central_interval
        self.use_roxzone = use_roxzone
        self.test_size = test_size
        self.random_state = random_state
        self.min_rows_per_checkpoint = min_rows_per_checkpoint

        self.models_by_t: Dict[int, CheckpointModel] = {}

    def fit(self, df_raw: pd.DataFrame) -> "HyroxBayesianLivePredictor":
        """
        Train checkpoint models.

        Notes:
        - We explicitly force preprocessing output to be dense to avoid sparse errors:
          * OneHotEncoder(sparse_output=False) produces dense output :contentReference[oaicite:7]{index=7}
          * ColumnTransformer(sparse_threshold=0) forces dense stacking :contentReference[oaicite:8]{index=8}
        """
        df = df_raw.copy()

        for col in set(SEGMENT_ORDER + [TOTAL_COL, ROXZONE_COL]):
            if col in df.columns:
                df[col] = _to_seconds(df[col])


        df = df[df[TOTAL_COL].notna() & (df[TOTAL_COL] > 0)].copy()

        for c in CAT_COLS:
            if c not in df.columns:
                df[c] = "Unknown"
            df[c] = df[c].astype("string").fillna("Unknown")

        for t in range(1, len(SEGMENT_ORDER) + 1):
            segments_used = SEGMENT_ORDER[:t]

            needed_cols = list(segments_used) + [TOTAL_COL] + CAT_COLS
            if self.use_roxzone and ROXZONE_COL in df.columns:
                needed_cols.append(ROXZONE_COL)

            df_t = df.dropna(subset=needed_cols).copy()
            if len(df_t) < self.min_rows_per_checkpoint:
                continue

            df_t["c_t"] = df_t[list(segments_used)].sum(axis=1)
            df_t["r_t"] = df_t[TOTAL_COL] - df_t["c_t"]
            df_t["n_segments"] = t

            df_t = df_t[(df_t["c_t"] > 0) & (df_t["r_t"] > 0)].copy()
            if len(df_t) < self.min_rows_per_checkpoint:
                continue

            num_cols = list(segments_used) + ["c_t", "n_segments"]
            if self.use_roxzone and ROXZONE_COL in df_t.columns:
                num_cols.append(ROXZONE_COL)

            X_raw = df_t[num_cols + CAT_COLS]
            y = df_t["r_t"].astype(float)

            pre = ColumnTransformer(
                transformers=[
                    ("num", StandardScaler(), num_cols),
                    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), CAT_COLS),
                ],
                remainder="drop",
                sparse_threshold=0.0,
            )

            X = pre.fit_transform(X_raw)

            X_train, X_val, y_train, y_val = train_test_split(
                X, y, test_size=self.test_size, random_state=self.random_state
            )

            br = BayesianRidge()
            br.fit(X_train, y_train)

            self.models_by_t[t] = CheckpointModel(
                preprocessor=pre,
                model=br,
                segments_used=tuple(segments_used),
                use_roxzone=self.use_roxzone,
            )

        if not self.models_by_t:
            raise ValueError("No checkpoint models were trained. Check time parsing / missing values / min_rows_per_checkpoint.")

        return self

    def predict_live(
        self,
        row: pd.Series,
        completed_segments: Sequence[str],
        *,
        central_interval: Optional[float] = None,
    ) -> Tuple[float, float, float]:
        """
        Predict finish time given a partially-completed race.

        completed_segments: list of segment column names the athlete has completed so far.
        IMPORTANT: this function assumes you're completing segments IN ORDER.
        If you pass a set, we'll take the longest prefix of SEGMENT_ORDER that is fully present.

        Returns: (mean, low, high) in seconds.
        """
        if central_interval is None:
            central_interval = self.central_interval

        completed_set = set(completed_segments)

        # Find the latest checkpoint t such that all segments up to t are completed
        t_best = 0
        for t in range(1, len(SEGMENT_ORDER) + 1):
            prefix = SEGMENT_ORDER[:t]
            if all(seg in completed_set for seg in prefix):
                t_best = t
            else:
                break

        if t_best == 0:
            raise ValueError("No valid checkpoint found. Provide at least the first segment in SEGMENT_ORDER (run1_time).")

        # If we didn't train a model exactly at t_best (e.g. skipped due to too few rows),
        # fall back to the closest earlier trained model.
        available_ts = sorted(self.models_by_t.keys())
        t_use = max([t for t in available_ts if t <= t_best])

        model = self.models_by_t[t_use]

        # Make sure the row has required columns
        missing = [c for c in model.segments_used if c not in row.index]
        if missing:
            raise ValueError(f"Row is missing required segment columns: {missing}")

        # Predict
        return model.predict_finish_seconds(row=row, central_interval=central_interval)

    def available_checkpoints(self) -> List[int]:
        """Which checkpoints we successfully trained (1..16 subset)."""
        return sorted(self.models_by_t.keys())


In [ ]:
df = s8.copy()
# df = your Season 8 dataframe with the columns you pasted
pred = HyroxBayesianLivePredictor(central_interval=0.80, use_roxzone=True)
pred.fit(df)

print("Trained checkpoints:", pred.available_checkpoints())

# Pick a row (in real life you'd build a row from live inputs)
row = df.iloc[0].copy()

# Suppose the athlete has completed: run1, ski, run2, sledPush
completed = ["run1_time", "skiErg_time", "run2_time", "sledPush_time"]

mean_s, low_s, high_s = pred.predict_live(row=row, completed_segments=completed)

print("Predicted finish (seconds):", mean_s)
print("80% interval (seconds):", (low_s, high_s))


In [ ]:
df = s8.copy()
df.loc[df['name'].str.contains('Matei')]


,age_group,division,event_id,event_name,gender,name,nationality,roxzone_time,run1_time,run2_time,...,total_time,skiErg_time,sledPush_time,sledPull_time,burpeeBroadJump_time,rowErg_time,farmersCarry_time,sandbagLunges_time,wallBalls_time,work_time
28474,40-44,pro_doubles,LR3MS4JIE12,2025 Birmingham,female,"Karen Cookes, Alexandra Mateita","GBR, ROU",6.233333,5.583333,4.883333,...,80.883333,4.650000,2.133333,4.750000,3.633333,5.083333,2.116667,4.383333,4.883333,31.633333
95869,30-34,doubles,LR3MS4JIDEA,2025 Geneva,female,"Mateille Laura, Audrey Fiorello","SUI, SUI",6.333333,6.600000,5.383333,...,82.383333,5.100000,1.916667,4.650000,4.050000,5.166667,1.450000,3.716667,4.500000,30.550000
109806,35-39,doubles,LR3MS4JIE8B,2025 Frankfurt,male,"Matei Coltan, Simon Attia","ROU, ISR",5.733333,3.700000,4.783333,...,68.616667,3.633333,2.316667,3.150000,2.616667,4.216667,1.466667,2.816667,3.950000,24.166667
132669,25-29,doubles,LR3MS4JI1262,2026 Manchester,mixed,"Kate Russell, Vlad Matei","GBR, ROU",3.650000,3.300000,4.533333,...,68.033333,4.100000,2.283333,3.800000,2.983333,4.483333,2.083333,3.766667,5.766667,29.266667
142001,25-29,open,LR3MS4JI10E2,2025 London Excel,male,"Matei, Vlad",ROU,9.066667,2.433333,2.950000,...,66.750000,4.566667,2.883333,4.433333,3.483333,4.666667,2.333333,4.316667,5.466667,32.150000
248509,40-44,doubles,LR3MS4JI124C,2026 Phoenix,female,"Miriam Matei, Chrystal Thomas","ROU, USA",13.883333,11.633333,11.916667,...,137.216667,5.900000,2.516667,6.466667,7.800000,6.400000,2.466667,6.000000,6.283333,43.833333
269275,35-39,pro_doubles,LR3MS4JIEA0,2025 Stuttgart,male,"Paul Matei Pop, Daniel Kraus","ROU, GER",14.400000,5.216667,5.666667,...,101.266667,4.050000,3.400000,6.900000,4.066667,4.800000,1.750000,5.083333,7.850000,37.900000


In [10]:
manchester_idx = 132669
london_idx = 142001

man_row = df.iloc[manchester_idx].copy()
lon_row = df.iloc[london_idx].copy()

In [52]:
up_to_idx = 15
SEGMENT_ORDER[:up_to_idx]

['run1_time',
 'skiErg_time',
 'run2_time',
 'sledPush_time',
 'run3_time',
 'sledPull_time',
 'run4_time',
 'burpeeBroadJump_time',
 'run5_time',
 'rowErg_time',
 'run6_time',
 'farmersCarry_time',
 'run7_time',
 'sandbagLunges_time',
 'run8_time']

In [53]:
mean_s, low_s, high_s = pred.predict_live(row=man_row, completed_segments=SEGMENT_ORDER[:up_to_idx])
print(f"mean: {mean_s}")
print(f"low: {low_s}")
print(f"high: {high_s}")

mean: 67.3387743403629
low: 63.1791227650156
high: 71.4984259157102


In [ ]:
# save the model
from pathlib import Path
import joblib

Path("models").mkdir(exist_ok=True)

joblib.dump(pred, "models/hyrox_live_predictor.joblib", compress=3)

In [6]:
# load the model
import joblib

pred = joblib.load("models/hyrox_live_predictor.joblib")